# Predicting sentiment from product reviews


The goal of this first notebook is to explore logistic regression and feature engineering with existing GraphLab functions.

In this notebook you will use product review data from Amazon.com to predict whether the sentiments about a product (from its reviews) are positive or negative.

* Use SFrames to do some feature engineering
* Train a logistic regression model to predict the sentiment of product reviews.
* Inspect the weights (coefficients) of a trained logistic regression model.
* Make a prediction (both class and probability) of sentiment for a new product review.
* Given the logistic regression weights, predictors and ground truth labels, write a function to compute the **accuracy** of the model.
* Inspect the coefficients of the logistic regression model and interpret their meanings.
* Compare multiple logistic regression models.

Let's get started!
    
## Fire up GraphLab Create

Make sure you have the latest version of GraphLab Create.

In [1]:
from __future__ import division
#import graphlab
import math
import string
import sframe

# Data preparation

We will use a dataset consisting of baby product reviews on Amazon.com.

In [2]:
products = sframe.SFrame('amazon_baby.gl/')

[INFO] sframe.cython.cy_server: SFrame v2.1 started. Logging /tmp/sframe_server_1543254391.log


Now, let us see a preview of what the dataset looks like.

In [3]:
products

name,review,rating
Planetwise Flannel Wipes,"These flannel wipes areOK, but in my opinion ...",3.0
Planetwise Wipe Pouch,it came early and was notdisappointed. i love ...,5.0
Annas Dream Full Quiltwith 2 Shams ...,Very soft and comfortableand warmer than it ...,5.0
Stop Pacifier Suckingwithout tears with ...,This is a product wellworth the purchase. I ...,5.0
Stop Pacifier Suckingwithout tears with ...,All of my kids have criednon-stop when I tried to ...,5.0
Stop Pacifier Suckingwithout tears with ...,"When the Binky Fairy cameto our house, we didn't ...",5.0
A Tale of Baby's Dayswith Peter Rabbit ...,"Lovely book, it's boundtightly so you may no ...",4.0
"Baby Tracker&reg; - DailyChildcare Journal, ...",Perfect for new parents.We were able to keep ...,5.0
"Baby Tracker&reg; - DailyChildcare Journal, ...",A friend of mine pinnedthis product on Pinte ...,5.0
"Baby Tracker&reg; - DailyChildcare Journal, ...",This has been an easy wayfor my nanny to record ...,4.0


## Build the word count vector for each review

Let us explore a specific example of a baby product.


In [4]:
products[269]

{'name': 'The First Years Massaging Action Teether',
 'rating': 5.0,
 'review': 'A favorite in our house!'}

Now, we will perform 2 simple data transformations:

1. Remove punctuation using [Python's built-in](https://docs.python.org/2/library/string.html) string functionality.
2. Transform the reviews into word-counts.

**Aside**. In this notebook, we remove all punctuations for the sake of simplicity. A smarter approach to punctuations would preserve phrases such as "I'd", "would've", "hadn't" and so forth. See [this page](https://www.cis.upenn.edu/~treebank/tokenization.html) for an example of smart handling of punctuations.

In [5]:
def remove_punctuation(text):
    import string
    return text.translate(None, string.punctuation) 

In [6]:
products['review_clean'] = products['review'].apply(remove_punctuation)

In [7]:
products[269]['review_clean']

'A favorite in our house'

In [10]:
def word_counter(text):
    word_count = {}
    lower_text = text.lower()
    for word in lower_text.split():
        if word not in word_count:
            word_count[word] = 1
        else:
            word_count[word] += 1
    return(word_count)

In [14]:
word_counter(products[269]['review_clean'])

{'a': 1, 'favorite': 1, 'house': 1, 'in': 1, 'our': 1}

In [18]:
#review_without_punctuation = products['review'].apply(remove_punctuation)
products['word_count'] = products['review_clean'].apply(word_counter)

In [19]:
products[269]['word_count']

{'a': 1, 'favorite': 1, 'house': 1, 'in': 1, 'our': 1}

Now, let us explore what the sample example above looks like after these 2 transformations. Here, each entry in the **word_count** column is a dictionary where the key is the word and the value is a count of the number of times the word occurs.

## Extract sentiments

We will **ignore** all reviews with *rating = 3*, since they tend to have a neutral sentiment.

In [20]:
products = products[products['rating'] != 3]
len(products)

166752

Now, we will assign reviews with a rating of 4 or higher to be *positive* reviews, while the ones with rating of 2 or lower are *negative*. For the sentiment column, we use +1 for the positive class label and -1 for the negative class label.

In [21]:
products['sentiment'] = products['rating'].apply(lambda rating : +1 if rating > 3 else -1)
products

name,review,rating,review_clean
Planetwise Wipe Pouch,it came early and was notdisappointed. i love ...,5.0,it came early and was notdisappointed i love ...
Annas Dream Full Quiltwith 2 Shams ...,Very soft and comfortableand warmer than it ...,5.0,Very soft and comfortableand warmer than it ...
Stop Pacifier Suckingwithout tears with ...,This is a product wellworth the purchase. I ...,5.0,This is a product wellworth the purchase I ...
Stop Pacifier Suckingwithout tears with ...,All of my kids have criednon-stop when I tried to ...,5.0,All of my kids have criednonstop when I tried to ...
Stop Pacifier Suckingwithout tears with ...,"When the Binky Fairy cameto our house, we didn't ...",5.0,When the Binky Fairy cameto our house we didnt ...
A Tale of Baby's Dayswith Peter Rabbit ...,"Lovely book, it's boundtightly so you may no ...",4.0,Lovely book its boundtightly so you may no ...
"Baby Tracker&reg; - DailyChildcare Journal, ...",Perfect for new parents.We were able to keep ...,5.0,Perfect for new parentsWe were able to keep ...
"Baby Tracker&reg; - DailyChildcare Journal, ...",A friend of mine pinnedthis product on Pinte ...,5.0,A friend of mine pinnedthis product on Pinte ...
"Baby Tracker&reg; - DailyChildcare Journal, ...",This has been an easy wayfor my nanny to record ...,4.0,This has been an easy wayfor my nanny to record ...
"Baby Tracker&reg; - DailyChildcare Journal, ...",I love this journal andour nanny uses it ...,4.0,I love this journal andour nanny uses it ...


Now, we can see that the dataset contains an extra column called **sentiment** which is either positive (+1) or negative (-1).

## Split data into training and test sets

Let's perform a train/test split with 80% of the data in the training set and 20% of the data in the test set. We use `seed=1` so that everyone gets the same result.

In [22]:
train_data, test_data = products.random_split(.8, seed=1)
print len(train_data)
print len(test_data)

133416
33336


# Train a sentiment classifier with logistic regression

We will now use logistic regression to create a sentiment classifier on the training data. This model will use the column **word_count** as a feature and the column **sentiment** as the target. We will use `validation_set=None` to obtain same results as everyone else.

**Note:** This line may take 1-2 minutes.

In [ ]:
#sentiment_model = graphlab.logistic_classifier.create(train_data,
#                                                      target = 'sentiment',
#                                                      features=['word_count'],
#                                                      validation_set=None)

In [25]:
from sklearn.linear_model import LogisticRegression

In [28]:
#sentiment_model = LogisticRegression(penalty=’l2’, dual=False, tol=0.0001, 
#                                     C=1.0, fit_intercept=True, intercept_scaling=1, 
#                                     class_weight=None, random_state=None, solver=’warn’, 
#                                     max_iter=100, multi_class=’warn’,
#                                     verbose=0, warm_start=False, n_jobs=None)
sentiment_model = LogisticRegression()
sentiment_model.fit(train_data['word_count'], train_data['sentiment'])

/home/mwyman/sandbox/virtenv/gl-env/lib/python2.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


TypeError: float() argument must be a string or a number

In [30]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(token_pattern=r'\b\w+\b')
     # Use this token pattern to keep single-letter words
# First, learn vocabulary from the training data and assign columns to words
# Then convert the training data into a sparse matrix
train_matrix = vectorizer.fit_transform(train_data['review_clean'])
# Second, convert the test data into a sparse matrix, using the same word-column mapping
test_matrix = vectorizer.transform(test_data['review_clean'])

In [31]:
sentiment_model = LogisticRegression()
sentiment_model.fit(train_matrix, train_data['sentiment'])

/home/mwyman/sandbox/virtenv/gl-env/lib/python2.7/site-packages/sklearn/svm/base.py:922: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [45]:
len(sentiment_model.coef_[0])

121712

In [51]:
import numpy as np
coeffs = np.asarray(sentiment_model.coef_[0])

In [53]:
sum(coeffs >= 0)

85974

**Aside**. You may get a warning to the effect of "Terminated due to numerical difficulties --- this model may not be ideal". It means that the quality metric (to be covered in Module 3) failed to improve in the last iteration of the run. The difficulty arises as the sentiment model puts too much weight on extremely rare words. A way to rectify this is to apply regularization, to be covered in Module 4. Regularization lessens the effect of extremely rare words. For the purpose of this assignment, however, please proceed with the model above.

Now that we have fitted the model, we can extract the weights (coefficients) as an SFrame as follows:

In [55]:
weights = sentiment_model.coef_[0]
weights.column_names()

AttributeError: 'numpy.ndarray' object has no attribute 'column_names'

There are a total of `121713` coefficients in the model. Recall from the lecture that positive weights $w_j$ correspond to weights that cause positive sentiment, while negative weights correspond to negative sentiment. 

Fill in the following block of code to calculate how many *weights* are positive ( >= 0). (**Hint**: The `'value'` column in SFrame *weights* must be positive ( >= 0)).

In [ ]:
num_positive_weights = ...
num_negative_weights = ...

print "Number of positive weights: %s " % num_positive_weights
print "Number of negative weights: %s " % num_negative_weights

**Quiz Question:** How many weights are >= 0?

## Making predictions with logistic regression

Now that a model is trained, we can make predictions on the **test data**. In this section, we will explore this in the context of 3 examples in the test dataset.  We refer to this set of 3 examples as the **sample_test_data**.

In [56]:
sample_test_data = test_data[10:13]
print sample_test_data['rating']
sample_test_data

[5.0, 2.0, 1.0]


Columns:
	name	str
	review	str
	rating	float
	review_clean	str
	word_count	dict
	sentiment	int

Rows: 3

Data:
+-------------------------------+-------------------------------+--------+
|              name             |             review            | rating |
+-------------------------------+-------------------------------+--------+
|   Our Baby Girl Memory Book   | Absolutely love it and all... |  5.0   |
| Wall Decor Removable Decal... | Would not purchase again o... |  2.0   |
| New Style Trailing Cherry ... | Was so excited to get this... |  1.0   |
+-------------------------------+-------------------------------+--------+
+-------------------------------+-------------------------------+-----------+
|          review_clean         |           word_count          | sentiment |
+-------------------------------+-------------------------------+-----------+
| Absolutely love it and all... | {'and': 2, 'all': 1, 'love... |     1     |
| Would not purchase again o... | {'and': 1, 'almost': 1, 'd... |     -1    |
| Was so excited to get this... | {'all': 1, 'money': 1, 'is... |     -1    |
+-------------------------------+-------------------------------+-----------+
[3 rows x 6 columns]

Let's dig deeper into the first row of the **sample_test_data**. Here's the full review:

In [57]:
sample_test_data[0]['review']

'Absolutely love it and all of the Scripture in it.  I purchased the Baby Boy version for my grandson when he was born and my daughter-in-law was thrilled to receive the same book again.'

That review seems pretty positive.

Now, let's see what the next row of the **sample_test_data** looks like. As we could guess from the sentiment (-1), the review is quite negative.

In [58]:
sample_test_data[1]['review']

'Would not purchase again or recommend. The decals were thick almost plastic like and were coming off the wall as I was applying them! The would NOT stick! Literally stayed stuck for about 5 minutes then started peeling off.'

We will now make a **class** prediction for the **sample_test_data**. The `sentiment_model` should predict **+1** if the sentiment is positive and **-1** if the sentiment is negative. Recall from the lecture that the **score** (sometimes called **margin**) for the logistic regression model  is defined as:

$$
\mbox{score}_i = \mathbf{w}^T h(\mathbf{x}_i)
$$ 

where $h(\mathbf{x}_i)$ represents the features for example $i$.  We will write some code to obtain the **scores** using GraphLab Create. For each row, the **score** (or margin) is a number in the range **[-inf, inf]**.

In [ ]:
scores = sentiment_model.predict(sample_test_data, output_type='margin')
print scores

In [59]:
sample_test_matrix = vectorizer.transform(sample_test_data['review_clean'])
scores = sentiment_model.decision_function(sample_test_matrix)
print scores

[  5.60582004  -3.1479641  -10.42485422]


### Predicting sentiment

These scores can be used to make class predictions as follows:

$$
\hat{y} = 
\left\{
\begin{array}{ll}
      +1 & \mathbf{w}^T h(\mathbf{x}_i) > 0 \\
      -1 & \mathbf{w}^T h(\mathbf{x}_i) \leq 0 \\
\end{array} 
\right.
$$

Using scores, write code to calculate $\hat{y}$, the class predictions:

In [65]:
def pred(score):
    if score > 0:
        sentiment = 1
    else:
        sentiment = -1
    return(sentiment)

for i in scores:
    print(pred(i))

1
-1
-1


Run the following code to verify that the class predictions obtained by your calculations are the same as that obtained from GraphLab Create.

In [62]:
print "Class predictions according to scikit-learn:" 
print sentiment_model.predict(sample_test_matrix)

Class predictions according to scikit-learn:
[ 1 -1 -1]


**Checkpoint**: Make sure your class predictions match with the one obtained from GraphLab Create.

### Probability predictions

Recall from the lectures that we can also calculate the probability predictions from the scores using:
$$
P(y_i = +1 | \mathbf{x}_i,\mathbf{w}) = \frac{1}{1 + \exp(-\mathbf{w}^T h(\mathbf{x}_i))}.
$$

Using the variable **scores** calculated previously, write code to calculate the probability that a sentiment is positive using the above formula. For each row, the probabilities should be a number in the range **[0, 1]**.

In [66]:
def prob_pred(score):
    return 1/(1+math.exp(-score))

for i in scores:
    print(prob_pred(i))

0.996337061949
0.0411715733413
2.96845484816e-05


**Checkpoint**: Make sure your probability predictions match the ones obtained from GraphLab Create.

In [68]:
print "Class predictions according to scikit-learn:" 
print sentiment_model.predict_proba(sample_test_matrix)

Class predictions according to scikit-learn:
[[3.66293805e-03 9.96337062e-01]
 [9.58828427e-01 4.11715733e-02]
 [9.99970315e-01 2.96845485e-05]]


** Quiz Question:** Of the three data points in **sample_test_data**, which one (first, second, or third) has the **lowest probability** of being classified as a positive review?

# Find the most positive (and negative) review

We now turn to examining the full test dataset, **test_data**, and use GraphLab Create to form predictions on all of the test data points for faster performance.

Using the `sentiment_model`, find the 20 reviews in the entire **test_data** with the **highest probability** of being classified as a **positive review**. We refer to these as the "most positive reviews."

To calculate these top-20 reviews, use the following steps:
1.  Make probability predictions on **test_data** using the `sentiment_model`. (**Hint:** When you call `.predict` to make predictions on the test data, use option `output_type='probability'` to output the probability rather than just the most likely class.)
2.  Sort the data according to those predictions and pick the top 20. (**Hint:** You can use the `.topk` method on an SFrame to find the top k rows sorted according to the value of a specified column.)

In [85]:
test_matrix = vectorizer.transform(test_data['review_clean'])
#test_scores = sentiment_model.decision_function(test_matrix)
test_pred = sentiment_model.predict_proba(test_matrix)

In [99]:
test_pred

array([[2.17630404e-01, 7.82369596e-01],
       [7.44361184e-07, 9.99999256e-01],
       [6.64622881e-02, 9.33537712e-01],
       ...,
       [5.34760045e-06, 9.99994652e-01],
       [2.54381908e-06, 9.99997456e-01],
       [1.88071859e-02, 9.81192814e-01]])

In [105]:
test_pred[0,1]

0.7823695960023087

In [110]:
pos_pred = test_pred[:,1]
pos_pred
neg_pred = test_pred[:,0]
neg_pred

array([2.17630404e-01, 7.44361184e-07, 6.64622881e-02, ...,
       5.34760045e-06, 2.54381908e-06, 1.88071859e-02])

In [111]:
pos_pred

array([0.7823696 , 0.99999926, 0.93353771, ..., 0.99999465, 0.99999746,
       0.98119281])

In [112]:
pranks = np.argsort(pos_pred)

In [113]:
pranks

array([ 2931, 21700, 13939, ..., 15732, 14482, 20743])

In [95]:
nranks = np.argsort(neg_pred)

**Quiz Question**: Which of the following products are represented in the 20 most positive reviews? [multiple choice]


Now, let us repeat this exercise to find the "most negative reviews." Use the prediction probabilities to find the  20 reviews in the **test_data** with the **lowest probability** of being classified as a **positive review**. Repeat the same steps above but make sure you **sort in the opposite order**.

In [106]:
# most positively reviewed
for i in range(20):
    print(test_data[nranks[i]]['name'], test_pred[nranks[i],1])

("Graco Pack 'n Play Element Playard - Flint", 1.0)
('Infantino Wrap and Tie Baby Carrier, Black Blueberries', 1.0)
('Baby Jogger City Mini GT Single Stroller, Shadow/Orange', 1.0)
('Baby Einstein Around The World Discovery Center', 1.0)
('Evenflo 6 Pack Classic Glass Bottle, 4-Ounce', 1.0)
('Diono RadianRXT Convertible Car Seat, Plum', 1.0)
('Roan Rocco Classic Pram Stroller 2-in-1 with Bassinet and Seat Unit - Coffee', 1.0)
('Britax 2012 B-Agile Stroller, Red', 1.0)
("P'Kolino Silly Soft Seating in Tias, Green", 1.0)
('Graco FastAction Fold Jogger Click Connect Stroller, Grapeade', 1.0)
('Simple Wishes Hands-Free Breastpump Bra, Pink, XS-L', 1.0)
("Fisher-Price Cradle 'N Swing,  My Little Snugabunny", 1.0)
('Freemie Hands-Free Concealable Breast Pump Collection System', 1.0)
('Evenflo X Sport Plus Convenience Stroller - Christina', 1.0)
('Mamas &amp; Papas 2014 Urbo2 Stroller - Black', 1.0)
('Buttons Cloth Diaper Cover - One Size - 8 Color Options', 1.0)
('Britax Decathlon Convertibl

**Quiz Question**: Which of the following products are represented in the 20 most negative reviews?  [multiple choice]

In [114]:
# most negatively reviewed
for i in range(20):
    print(test_data[pranks[i]]['name'], test_pred[pranks[i],0])

('Fisher-Price Ocean Wonders Aquarium Bouncer', 0.9999999999999991)
("Levana Safe N'See Digital Video Baby Monitor with Talk-to-Baby Intercom and Lullaby Control (LV-TW501)", 0.9999999999999981)
('Safety 1st Exchangeable Tip 3 in 1 Thermometer', 0.9999999999999191)
('Adiri BPA Free Natural Nurser Ultimate Bottle Stage 1 White, Slow Flow (0-3 months)', 0.9999999999998576)
('VTech Communications Safe &amp; Sounds Full Color Video and Audio Monitor', 0.9999999999997994)
('The First Years True Choice P400 Premium Digital Monitor, 2 Parent Unit', 0.9999999999995248)
('Safety 1st High-Def Digital Monitor', 0.9999999999655913)
('Cloth Diaper Sprayer--styles may vary', 0.9999999999614754)
('Motorola Digital Video Baby Monitor with Room Temperature Thermometer', 0.9999999999075713)
('Philips AVENT Newborn Starter Set', 0.9999999998966123)
('Cosco Alpha Omega Elite Convertible Car Seat', 0.9999999995649413)
('Ellaroo Mei Tai Baby Carrier - Hershey', 0.999999999537765)
('Belkin WeMo Wi-Fi Baby Mo

In [115]:
test_data[2931]['review']

"We have not had ANY luck with Fisher-Price products.  When this P.O.S. worked for about 6 short weeks though, it was a good product.  Ours broke without warning, and now the aquarium part does not work at all. We get music, sound effects, and the blinking lights, but no bubbles and fish/starfish movement.  It really sucks to see (and hear) the dissapointment on our baby's face as the aquarium suddenly stopped working!  Also, our other bouncer, a Fisher-Price 'Flutter & Chime' bouncer, has this plastic bird that vibrates excessively and makes a loud, annoying noise.  It is not affixed properly to the product to begin with.  We also were dissapointed in the Fisher-Price 'Aquarium Cradle Swing', since the swing goes much too fast, even on the slowest setting.  All in all, 3 different products by Fisher-Price, and all fell short in expectations and useability.  Bottom line- look elsewhere for a better quality bouncer.  Clearly, Fisher-Price quality is not too good on many of their seeming

## Compute accuracy of the classifier

We will now evaluate the accuracy of the trained classifier. Recall that the accuracy is given by


$$
\mbox{accuracy} = \frac{\mbox{# correctly classified examples}}{\mbox{# total examples}}
$$

This can be computed as follows:

* **Step 1:** Use the trained model to compute class predictions (**Hint:** Use the `predict` method)
* **Step 2:** Count the number of data points when the predicted class labels match the ground truth labels (called `true_labels` below).
* **Step 3:** Divide the total number of correct predictions by the total number of data points in the dataset.

Complete the function below to compute the classification accuracy:

In [116]:
def get_classification_accuracy(model, data, true_labels):
    # First get the predictions
    pred = model.predict(data)
        
    # Compute the number of correctly classified examples
    num_correct = sum(np.asarray(pred == true_labels))
    

    # Then compute accuracy by dividing num_correct by total number of examples
    total_samples = len(true_labels)
    accuracy = num_correct / total_samples
        
    return accuracy

Now, let's compute the classification accuracy of the **sentiment_model** on the **test_data**.

In [117]:
get_classification_accuracy(sentiment_model, test_matrix, test_data['sentiment'])

0.9322654187664987

**Quiz Question**: What is the accuracy of the **sentiment_model** on the **test_data**? Round your answer to 2 decimal places (e.g. 0.76).

**Quiz Question**: Does a higher accuracy value on the **training_data** always imply that the classifier is better?

## Learn another classifier with fewer words

There were a lot of words in the model we trained above. We will now train a simpler logistic regression model using only a subset of words that occur in the reviews. For this assignment, we selected a 20 words to work with. These are:

In [118]:
significant_words = ['love', 'great', 'easy', 'old', 'little', 'perfect', 'loves', 
      'well', 'able', 'car', 'broke', 'less', 'even', 'waste', 'disappointed', 
      'work', 'product', 'money', 'would', 'return']

In [119]:
len(significant_words)

20

For each review, we will use the **word_count** column and trim out all words that are **not** in the **significant_words** list above. We will use the [SArray dictionary trim by keys functionality]( https://dato.com/products/create/docs/generated/graphlab.SArray.dict_trim_by_keys.html). Note that we are performing this on both the training and test set.

In [120]:
train_data['word_count_subset'] = train_data['word_count'].dict_trim_by_keys(significant_words, exclude=False)
test_data['word_count_subset'] = test_data['word_count'].dict_trim_by_keys(significant_words, exclude=False)

Let's see what the first example of the dataset looks like:

In [121]:
train_data[0]['review']

'it came early and was not disappointed. i love planet wise bags and now my wipe holder. it keps my osocozy wipes moist and does not leak. highly recommend it.'

The **word_count** column had been working with before looks like the following:

In [122]:
print train_data[0]['word_count']

{'and': 3, 'love': 1, 'it': 3, 'highly': 1, 'osocozy': 1, 'bags': 1, 'leak': 1, 'moist': 1, 'does': 1, 'recommend': 1, 'was': 1, 'wipes': 1, 'disappointed': 1, 'early': 1, 'not': 2, 'now': 1, 'holder': 1, 'wipe': 1, 'keps': 1, 'wise': 1, 'i': 1, 'planet': 1, 'my': 2, 'came': 1}


Since we are only working with a subset of these words, the column **word_count_subset** is a subset of the above dictionary. In this example, only 2 `significant words` are present in this review.

In [123]:
print train_data[0]['word_count_subset']

{'disappointed': 1, 'love': 1}


## Train a logistic regression model on a subset of data

We will now build a classifier with **word_count_subset** as the feature and **sentiment** as the target. 

In [124]:
vectorizer_word_subset = CountVectorizer(vocabulary=significant_words) # limit to 20 words
train_matrix_word_subset = vectorizer_word_subset.fit_transform(train_data['review_clean'])
test_matrix_word_subset = vectorizer_word_subset.transform(test_data['review_clean'])

In [149]:
simple_model = LogisticRegression()
simple_model.fit(train_matrix_word_subset, train_data['sentiment'])

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

We can compute the classification accuracy using the `get_classification_accuracy` function you implemented earlier.

In [150]:
get_classification_accuracy(simple_model, test_matrix_word_subset, test_data['sentiment'])

0.8693604511639069

Now, we will inspect the weights (coefficients) of the **simple_model**:

In [151]:
simple_model.coef_

array([[ 1.36368976,  0.94399959,  1.19253827,  0.08551278,  0.52018576,
         1.50981248,  1.67307389,  0.50376046,  0.19090857,  0.05885467,
        -1.65157634, -0.20956286, -0.51137963, -2.03369861, -2.34829822,
        -0.62116877, -0.32055624, -0.89803074, -0.36216674, -2.10933109]])

Let's sort the coefficients (in descending order) by the **value** to obtain the coefficients with the most positive effect on the sentiment.

In [152]:
simple_coeffs = simple_model.coef_[0]

**Quiz Question**: Consider the coefficients of **simple_model**. There should be 21 of them, an intercept term + one for each word in **significant_words**. How many of the 20 coefficients (corresponding to the 20 **significant_words** and *excluding the intercept term*) are positive for the `simple_model`?

In [153]:
sum(simple_coeffs > 0)

10

In [154]:
simple_coeffs

array([ 1.36368976,  0.94399959,  1.19253827,  0.08551278,  0.52018576,
        1.50981248,  1.67307389,  0.50376046,  0.19090857,  0.05885467,
       -1.65157634, -0.20956286, -0.51137963, -2.03369861, -2.34829822,
       -0.62116877, -0.32055624, -0.89803074, -0.36216674, -2.10933109])

In [144]:
print(simple_coeffs)

[-2.34829822 -2.10933109 -2.03369861 -1.65157634 -0.89803074 -0.62116877
 -0.51137963 -0.36216674 -0.32055624 -0.20956286  0.05885467  0.08551278
  0.19090857  0.50376046  0.52018576  0.94399959  1.19253827  1.36368976
  1.50981248  1.67307389]


**Quiz Question**: Are the positive words in the **simple_model** (let us call them `positive_significant_words`) also positive words in the **sentiment_model**?

In [156]:
significant_words[[simple_coeffs > 0]]

TypeError: list indices must be integers, not list

In [157]:
sig_words = np.asarray(significant_words)

In [160]:
positive_significant_words = sig_words[simple_coeffs > 0]

In [161]:
positive_significant_words

array(['love', 'great', 'easy', 'old', 'little', 'perfect', 'loves',
       'well', 'able', 'car'], dtype='|S12')

In [ ]:
sig_words_loc 

In [159]:
sig_words[simple_coeffs < 0]

array(['broke', 'less', 'even', 'waste', 'disappointed', 'work',
       'product', 'money', 'would', 'return'], dtype='|S12')

In [162]:
simple_model_coef_table = sframe.SFrame({'word':significant_words,
                                         'coefficient':simple_model.coef_.flatten()})

In [164]:
simple_model_coef_table.print_rows(num_rows = 20)

+-----------------+--------------+
|   coefficient   |     word     |
+-----------------+--------------+
|  1.36368975931  |     love     |
|  0.943999590572 |    great     |
|  1.19253827349  |     easy     |
|  0.085512779463 |     old      |
|  0.520185762718 |    little    |
|  1.50981247669  |   perfect    |
|  1.67307389259  |    loves     |
|  0.503760457768 |     well     |
|  0.190908572064 |     able     |
| 0.0588546711528 |     car      |
|  -1.65157634497 |    broke     |
| -0.209562864535 |     less     |
| -0.511379631799 |     even     |
|  -2.03369861394 |    waste     |
|  -2.3482982195  | disappointed |
| -0.621168773642 |     work     |
| -0.320556236735 |   product    |
| -0.898030737715 |    money     |
| -0.362166742274 |    would     |
|  -2.10933109032 |    return    |
+-----------------+--------------+
[20 rows x 2 columns]



In [167]:
vocab = np.asarray(vectorizer.get_feature_names())

In [187]:
for i in significant_words:
    print(i, coeffs[np.argwhere(vocab == i)[0]] )
    

('love', array([1.57301532]), 1.363689759311003)
('great', array([1.22507914]), 1.363689759311003)
('easy', array([1.35195263]), 1.363689759311003)
('old', array([0.0548859]), 1.363689759311003)
('little', array([0.63885269]), 1.363689759311003)
('perfect', array([1.8627836]), 1.363689759311003)
('loves', array([1.51794273]), 1.363689759311003)
('well', array([0.54122006]), 1.363689759311003)
('able', array([0.38996863]), 1.363689759311003)
('car', array([0.12444735]), 1.363689759311003)
('broke', array([-1.392657]), 1.363689759311003)
('less', array([-0.27680007]), 1.363689759311003)
('even', array([-0.46501272]), 1.363689759311003)
('waste', array([-1.99364495]), 1.363689759311003)
('disappointed', array([-2.19632713]), 1.363689759311003)
('work', array([-0.46178434]), 1.363689759311003)
('product', array([-0.1898968]), 1.363689759311003)
('money', array([-0.78568988]), 1.363689759311003)
('would', array([-0.28694459]), 1.363689759311003)
('return', array([-1.65555404]), 1.3636897593

In [180]:
len(coeffs)

121712

In [ ]:
sentimentel_model_coef_table = sframe.SFrame({'word':significant_words,
                                         'coefficient':sentimentel_model.coef_.flatten()})

# Comparing models

We will now compare the accuracy of the **sentiment_model** and the **simple_model** using the `get_classification_accuracy` method you implemented above.

First, compute the classification accuracy of the **sentiment_model** on the **train_data**:

In [175]:
get_classification_accuracy(sentiment_model, train_matrix, train_data['sentiment'])

0.9679123943155243

Now, compute the classification accuracy of the **simple_model** on the **train_data**:

In [172]:
get_classification_accuracy(simple_model, train_matrix_word_subset, train_data['sentiment'])

0.8668225700065959

**Quiz Question**: Which model (**sentiment_model** or **simple_model**) has higher accuracy on the TRAINING set?

Now, we will repeat this exercise on the **test_data**. Start by computing the classification accuracy of the **sentiment_model** on the **test_data**:

In [176]:
get_classification_accuracy(sentiment_model, test_matrix, test_data['sentiment'])

0.9322654187664987

Next, we will compute the classification accuracy of the **simple_model** on the **test_data**:

In [173]:
get_classification_accuracy(simple_model, test_matrix_word_subset, test_data['sentiment'])

0.8693604511639069

**Quiz Question**: Which model (**sentiment_model** or **simple_model**) has higher accuracy on the TEST set?

## Baseline: Majority class prediction

It is quite common to use the **majority class classifier** as the a baseline (or reference) model for comparison with your classifier model. The majority classifier model predicts the majority class for all data points. At the very least, you should healthily beat the majority class classifier, otherwise, the model is (usually) pointless.

What is the majority class in the **train_data**?

In [177]:
num_positive  = (train_data['sentiment'] == +1).sum()
num_negative = (train_data['sentiment'] == -1).sum()
print num_positive
print num_negative

112164
21252


Now compute the accuracy of the majority class classifier on **test_data**.

**Quiz Question**: Enter the accuracy of the majority class classifier model on the **test_data**. Round your answer to two decimal places (e.g. 0.76).

In [179]:
num_positive  = (test_data['sentiment'] == +1).sum()
num_negative = (test_data['sentiment'] == -1).sum()
print num_positive
print num_negative
num_positive/(num_positive + num_negative)

28095
5241


0.8427825773938085

**Quiz Question**: Is the **sentiment_model** definitely better than the majority class classifier (the baseline)?